# Importation of the librairies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import import_ipynb

from math import *
from functions import *

importing Jupyter notebook from functions.ipynb


# Extraction of needed astrophysical information 

In [2]:
l = countLigne("planets.csv")
df1 = pd.read_csv("planets.csv",skiprows=l-1)
columns = [
  "rowid",
 "pl_hostname",# COLUMN pl_hostname:    Host Name
 "pl_letter",# COLUMN pl_letter:      Planet Letter
 "pl_orbsmax",# COLUMN pl_orbsmax:     Orbit Semi-Major Axis [AU])
 "pl_radj",# COLUMN pl_radj:        Planet Radius [Jupiter radii]
 "pl_dens",# COLUMN pl_dens:        Planet Density [g/cm**3]
 "ra",# COLUMN ra:             RA [decimal degrees]
 "dec",# COLUMN dec:            Dec [decimal degrees]
 "st_dist",# COLUMN st_dist:        Distance [pc]
 "st_optmag",# COLUMN st_optmag:      Optical Magnitude [mag]
 "st_optband",# COLUMN st_optband:     Optical Magnitude Band
 "st_teff",# COLUMN st_teff:        Effective Temperature [K]
 "st_mass",# COLUMN st_mass:        Stellar Mass [Solar mass]
 "st_rad",# COLUMN st_rad:         Stellar Radius [Solar radii]
 "pl_angsep",# COLUMN pl_angsep:      Calculated Angular Separation [mas]
 "pl_eqt",# COLUMN pl_eqt:         Equilibrium Temperature [K]
 "pl_massj",# COLUMN pl_massj:       Planet Mass [Jupiter mass]
 "pl_facility",# COLUMN pl_facility:    Discovery Facility
 "st_wise1"]

df1.drop(columns, inplace=True, axis=1)
df1.columns

df2 = pd.read_csv("planets.csv",skiprows=l-1)
df2.drop(df1.columns, inplace=True, axis=1)

#  Naming of needed astrophysical values

In [3]:
Nr_p = df2.loc[:,'rowid'] #Numéro de la planète
W1   = df2.loc[:,'pl_hostname']
P_l = df2.loc[:,'pl_letter']
O_p = df2.loc[:,'pl_orbsmax']*AU #Orbite planetes(m)
M_p = df2.loc[:,'pl_massj']*MJ #Masse de la planète (kg)
R_p = df2.loc[:,'pl_radj']*RJ #Rayon de la planète (m)
T_p = df2.loc[:,'pl_eqt']    #Température planétaire en K
rho  = df2.loc[:,'pl_dens']*1e3 # Masse volumique de la planète en kilogrammes par mètres cubes 
P_ad  = df2.loc[:,'ra'] # Ascension droite des planètes en degrés décimaux
P_d   = df2.loc[:,'dec'] #Déclinaison des planètes en degrés décimaux
P_mt = df2.loc[:,'st_optband'] # Bande Magnitude apparente de l'étoile hôte
P_m = df2.loc[:,'st_optmag'] #Magnitude apparente de l'étoile hôte
m_L = df2.loc[:,"st_wise1"]#Magnitude apparente en Bande Wise 3.4µm (la plus proche de la bande L)
M_S = df2.loc[:,"st_mass"]*M_so #Masse des étoiles (kg)
D_s = df2.loc[:,"st_dist"]*pc #Distance stellaire en mètres
T_s = df2.loc[:,"st_teff"] 	 #Température effective stellaire en K
R_s = df2.loc[:,"st_rad"]*R_so #Rayon stellaire en mètres
M_n = df2.loc[:,"pl_facility"] # Méthodes de détection pour chaque découvertes
S_p = df2.loc[:,"pl_angsep"]*m2r*D_s  #Separation étoile-hôte exoplanète en mètres
S_pua = df2.loc[:,"pl_angsep"]*m2r*D_s/AU # Separation étoile hôte exoplanète en UA
S_pa = df2.loc[:,"pl_angsep"] #Separation étoile hôte exoplanète en mas

# Radius and Temperature recomputation

In [4]:
R_pr = Re_Computation_Planet_Radius(Nr_p,R_p,W1,M_p,rho)

In [5]:
T_pr = Re_Computation_Planet_Eff_Temp(Nr_p,T_p,R_s,S_p,T_s)

In [6]:
#for i in range(0,len(T_pr)):
#    print('{:3.2e}'.format(T_pr[i]),'\t','{:3.2e}'.format(T_p[i]))

In [7]:
D = {}
D[0]      = Nr_p
D[1]      = O_p
D[2]      = M_p
D[3]      = R_pr
D[4]      = T_pr
D[5]      = P_ad
D[6]      = P_d
D[7]      = P_l
D[8]      = P_mt
D[9]      = P_m
D[10]     = D_s
D[11]     = T_s
D[12]     = R_s
D[13]     = W1
D[15]     = S_p
D[16]     = S_pua
D[17]     = S_pa 

# Flux calculations

### Stellar flux

In [8]:
s =0
F_s = Flux_Stellar(L,T_s,R_s,D_s)
for i in range(0,len(F_s)):
    if math.isnan(F_s[i]) is True:
        s +=1
print(s)
print(len(D[0]))

300
3793


### Thermal planetary flux

In [9]:
s =0
F_p = Flux_Planet(L,Nr_p,T_pr,R_pr,D_s)
for i in range(0,len(F_p)):
    if math.isnan(F_p[i]) is True:
        s +=1
print(s)
print(len(D[0]))

2423
3793


functions.ipynb:8: RuntimeWarning: overflow encountered in exp
  "source": [


### Reflective planetary flux

In [10]:
F_rp  =  A_B * (R_pr**2)/(4*O_p**2) * F_s

### Total Flux

In [11]:
F = F_p + F_rp

# Integration time calculation(days)

In [12]:
Time = Integration_time(Nr_p,F)

# Contrast planet-star calculation

In [13]:
C = F/F_s

# Selection of the targets

In [14]:
#Mise a 0 du compteur
s=0 

fichier_1 = open("Targets_HI5.tbl","w")
fichier_2 = open("TARGETS_NAMES.tbl","w")

fichier_1.write("\n")
fichier_1.write( "Observation wavelength : " + str("%.2e" %L) + " m" )
fichier_1.write("\n")
fichier_1.write( "Bond Albedo: " + str("%.2e" %A_B) )
fichier_1.write("\n")
fichier_1.write( "The base line is: " + str("%.2e" %B_max) + " m" )
fichier_1.write("\n")
fichier_1.write("Flux is the thermal flux of the planet \nR flux is the reflective flux of the planet")
fichier_1.write("\n")
fichier_1.write("The contrast is computed for the total flux")

43

In [15]:
hem = 'NORTH'
Display_Stars(D,df2,C,Time,fichier_1,hem)
hem = 'SOUTH'
Display_Stars(D,df2,C,Time,fichier_1,hem)

In [16]:
hem = 'NORTH'
Display_Planets(D,C,Time,fichier_1,F_p,F_rp,hem)
hem = 'SOUTH'
Display_Planets(D,C,Time,fichier_1,F_p,F_rp,hem)

In [17]:
fichier_1.close()
fichier_2.close()

In [18]:
s = 0
for i in range(0,len(D[0])):
    if math.isnan(S_pa[i]) is True:
        df2.loc[i,"pl_angsep"] = 0
    if math.isnan(C[i]) is True:
        C[i] = 0
        s += 1
print('s =',s)

s = 2458


# Displaying the graphics

In [ ]:
plt.figure(figsize=(15, 10), dpi=100)

f =20

plt.title(u"Constrast vs Angular separation \n" + "$\lambda$ = " + str("%.2e"%L) + " m" ,fontsize = f)
plt.xlabel(u"Angular separartion (mas)",fontsize = f)
plt.ylabel(u"Contrast",fontsize = f)

plt.ylim(ymin=1e-10, ymax=1e-3)
plt.xlim(xmin = 1e-3,xmax=1e3)

plt.xscale('log')
plt.yscale('log')

plt.grid(True)

print("Don't panic ! It is computing...")

compteur = 0

for i in range(0,len(D[0])):
    
    if C[i]!=0 or df2.loc[i,"pl_angsep"] != 0:
        
        if C[i]>=C_lim and R <= D[17][i] <= OWA and Time[i]<T_max:
        
            #print(C[i], D[17][i])

            compteur += 1

            plt.plot(S_pa[i],C[i],'o',label= D[13][i] + '\t'+ D[7][i] )

        else :
            plt.plot(S_pa[i],C[i],'o',color='grey')

if compteur < 30:
    plt.legend(prop={'size':10})

plt.savefig('./Contrast_AngularResolution.png')
plt.show()